In [16]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import math

In [17]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
train_dir = os.path.join('./Data/Blood_Cell_Classification/images/TRAIN/')
test_dir = os.path.join('./Data/Blood_Cell_Classification/images/TEST/')

In [19]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=16, target_size=(220, 200), color_mode='rgb')

In [20]:
test_generator = test_datagen.flow_from_directory(test_dir, batch_size=16, target_size=(220, 200), color_mode='rgb')

In [21]:
input_tensor = Input(shape=(220, 200, 3), dtype='float32', name='input')

In [22]:
pre_trained_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(220, 200, 3))
pre_trained_vgg.trainable = False
pre_trained_vgg.summary()

In [23]:
additional_model = models.Sequential()
additional_model.add(pre_trained_vgg)
additional_model.add(layers.Flatten())
additional_model.add(layers.Dense(4096, activation='relu'))
additional_model.add(layers.Dense(2048, activation='relu'))
additional_model.add(layers.Dense(1024, activation='relu'))
additional_model.add(layers.Dense(4, activation='softmax'))
additional_model.summary()

In [15]:
additional_model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])
checkpoint = ModelCheckpoint(filepath='pretrained_VGG_weight.hdf5', 
            monitor='loss', 
            mode='min', 
            save_best_only=True)

history = additional_model.fit_generator(train_generator, 
            steps_per_epoch=math.ceil(train_generator.n / train_generator.batch_size), 
            epochs=300, 
            validation_data=test_generator, 
            validation_steps=math.ceil(test_generator.n / test_generator.batch_size), 
            use_multiprocessing=True,                             
            workers=8)

Epoch 1/300
623/623 [==============================] - 964s 2s/step - loss: 0.5432 - acc: 0.7911 - val_loss: 1.1546 - val_acc: 0.5340
Epoch 2/300
623/623 [==============================] - 976s 2s/step - loss: 0.3368 - acc: 0.8722 - val_loss: 1.4891 - val_acc: 0.5219
Epoch 3/300
623/623 [==============================] - 1001s 2s/step - loss: 0.2202 - acc: 0.9150 - val_loss: 1.3758 - val_acc: 0.5806
Epoch 4/300
623/623 [==============================] - 1031s 2s/step - loss: 0.1534 - acc: 0.9438 - val_loss: 1.6805 - val_acc: 0.5774
Epoch 5/300
201/623 [========>.....................] - ETA: 9:33 - loss: 0.1108 - acc: 0.9629

Process Keras_worker_ForkPoolWorker-82:
Process Keras_worker_ForkPoolWorker-87:
Process Keras_worker_ForkPoolWorker-85:
Process Keras_worker_ForkPoolWorker-84:
Process Keras_worker_ForkPoolWorker-83:
Process Keras_worker_ForkPoolWorker-86:
Process Keras_worker_ForkPoolWorker-88:
Process Keras_worker_ForkPoolWorker-81:
Traceback (most recent call last):
  File "/Home2/wbjin/.conda/envs/tf36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Home2/wbjin/.conda/envs/tf36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Home2/wbjin/.conda/envs/tf36/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Home2/wbjin/.conda/envs/tf36/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/Home2/wbjin/.conda/envs/tf36/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes